In [5]:
import pandas as pd
import requests
import os
import zipfile

diretorio_atual = os.getcwd()

os.chdir(f"{diretorio_atual}\dados_cvm")

In [7]:
anos = range(2010,2023)

url_base = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/"

for ano in anos:
    
    download = requests.get(url_base + f"dfp_cia_aberta_{ano}.zip")
    
    open(f"dfp_cia_aberta_{ano}.zip", "wb").write(download.content)

In [15]:
lista_demonstracoes_2010_2022 = []

diretorio_atual = os.getcwd()

for arquivo in os.listdir(diretorio_atual):

    if arquivo == ".ipynb_checkpoints":
    
        pass
    
    else:
        
        arquivo_zip = zipfile.ZipFile(arquivo)
        
        for planilha in arquivo_zip.namelist():
            
            demonstracao = pd.read_csv(arquivo_zip.open(planilha), sep = ";", encoding = 'ISO-8859-1',
                                      dtype = {"ORDEM_EXERC": "category"})
            
            lista_demonstracoes_2010_2022.append(demonstracao)

In [24]:
base_dados = pd.concat(lista_demonstracoes_2010_2022)

base_dados

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,CATEG_DOC,ID_DOC,DT_RECEB,LINK_DOC,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF
0,00.000.000/0001-91,2010-12-31,1,BCO BRASIL S.A.,1023,DFP,4822.0,2011-02-17,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00.000.000/0001-91,2010-12-31,2,BCO BRASIL S.A.,1023,DFP,4823.0,2011-02-17,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DFP,6945.0,2011-04-30,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00.000.208/0001-00,2010-12-31,1,BRB BCO DE BRASILIA S.A.,14206,DFP,5436.0,2011-03-22,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00.000.208/0001-00,2010-12-31,2,BRB BCO DE BRASILIA S.A.,14206,DFP,6775.0,2011-04-29,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41244,97.837.181/0001-47,2022-12-31,1,DEXCO S.A.,21091,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2022-12-31,7.08.04.02,Dividendos,0.0,S,2022-01-01,NaN
41245,97.837.181/0001-47,2022-12-31,1,DEXCO S.A.,21091,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2021-12-31,7.08.04.03,Lucros Retidos / Prejuízo do Período,847006.0,S,2021-01-01,NaN
41246,97.837.181/0001-47,2022-12-31,1,DEXCO S.A.,21091,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2022-12-31,7.08.04.03,Lucros Retidos / Prejuízo do Período,506861.0,S,2022-01-01,NaN
41247,97.837.181/0001-47,2022-12-31,1,DEXCO S.A.,21091,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2021-12-31,7.08.05,Outros,0.0,S,2021-01-01,NaN


In [25]:
base_dados[['con_ind', 'tipo_dem']] = base_dados['GRUPO_DFP'].str.split("-", expand = True)
base_dados['con_ind'] = base_dados['con_ind'].str.strip()
base_dados['tipo_dem'] = base_dados['tipo_dem'].str.strip()

base_dados = base_dados[base_dados['ORDEM_EXERC'] != "PENÚLTIMO"]

In [26]:
lista_dem = base_dados['tipo_dem'].unique()

In [42]:
lista_empresas = base_dados['DENOM_CIA'].unique()

bb_dre = base_dados[(base_dados["DENOM_CIA"] == "BCO BRASIL S.A.") &
                   (base_dados["tipo_dem"] == "Demonstração do Resultado") &
                   (base_dados["DS_CONTA"] == "Receitas da Intermediação Financeira") &
                   (base_dados["con_ind"] == "DF Consolidado")]

bb_dre2 = base_dados[(base_dados["DENOM_CIA"] == "BCO BRASIL S.A.") &
                   (base_dados["tipo_dem"] == "Demonstração do Resultado") &
                   (base_dados["DS_CONTA"] == "Receitas de Intermediação Financeira") &
                   (base_dados["con_ind"] == "DF Consolidado")]

bb_dre.append(bb_dre2)

C:\Users\PICHAU\AppData\Local\Temp\ipykernel_6588\1606983645.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bb_dre.append(bb_dre2)


,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,CATEG_DOC,ID_DOC,DT_RECEB,LINK_DOC,GRUPO_DFP,...,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF,con_ind,tipo_dem
1,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração do Resultado,...,ÚLTIMO,2010-12-31,3.01,Receitas da Intermediação Financeira,85143206.0,S,2010-01-01,NaN,DF Consolidado,Demonstração do Resultado
1,00.000.000/0001-91,2011-12-31,4,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração do Resultado,...,ÚLTIMO,2011-12-31,3.01,Receitas da Intermediação Financeira,106919760.0,S,2011-01-01,NaN,DF Consolidado,Demonstração do Resultado
1,00.000.000/0001-91,2012-12-31,4,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração do Resultado,...,ÚLTIMO,2012-12-31,3.01,Receitas da Intermediação Financeira,107931378.0,S,2012-01-01,NaN,DF Consolidado,Demonstração do Resultado
1,00.000.000/0001-91,2013-12-31,4,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração do Resultado,...,ÚLTIMO,2013-12-31,3.01,Receitas da Intermediação Financeira,104582211.0,S,2013-01-01,NaN,DF Consolidado,Demonstração do Resultado
1,00.000.000/0001-91,2014-12-31,2,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração do Resultado,...,ÚLTIMO,2014-12-31,3.01,Receitas da Intermediação Financeira,137778601.0,S,2014-01-01,NaN,DF Consolidado,Demonstração do Resultado
1,00.000.000/0001-91,2015-12-31,3,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração do Resultado,...,ÚLTIMO,2015-12-31,3.01,Receitas da Intermediação Financeira,182368871.0,S,2015-01-01,NaN,DF Consolidado,Demonstração do Resultado
1,00.000.000/0001-91,2016-12-31,3,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração do Resultado,...,ÚLTIMO,2016-12-31,3.01,Receitas da Intermediação Financeira,168039236.0,S,2016-01-01,NaN,DF Consolidado,Demonstração do Resultado
1,00.000.000/0001-91,2017-12-31,2,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração do Resultado,...,ÚLTIMO,2017-12-31,3.01,Receitas da Intermediação Financeira,139763614.0,S,2017-01-01,NaN,DF Consolidado,Demonstração do Resultado
1,00.000.000/0001-91,2018-12-31,2,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração do Resultado,...,ÚLTIMO,2018-12-31,3.01,Receitas da Intermediação Financeira,120250344.0,S,2018-01-01,NaN,DF Consolidado,Demonstração do Resultado
1,00.000.000/0001-91,2019-12-31,2,BCO BRASIL S.A.,1023,NaN,NaN,NaN,NaN,DF Consolidado - Demonstração do Resultado,...,ÚLTIMO,2019-12-31,3.01,Receitas da Intermediação Financeira,123105325.0,S,2019-01-01,NaN,DF Consolidado,Demonstração do Resultado
